In [1]:
import json
import os
import glob
def extract_schema_info(schema_file):
    """
    Parse the schema.json file and extract all table names and column names.
    :param schema_file: Path to the JSON file
    :return: A dictionary containing table names as keys and column names as values
    """
    with open(schema_file, "r", encoding="utf-8") as file:
        schema = json.load(file)

    schema_info = {}
    print(schema.get("wikidata_topic_item_id"))
    print(schema.get("wikidata_topic_item_label"))
    for table in schema.get("tables", []):
        table_name = table.get("table_name", "UNKNOWN_TABLE")  # Get table name
        column_names = [col["column_name"] for col in table.get("columns", [])]  # Extract column names
        schema_info[table_name] = column_names  # Store in dictionary

    return schema_info

def display_schema_info(schema_info):
    """
    Print the extracted schema information in a structured format.
    """
    for table, columns in schema_info.items():
        print(f"    ├── 📂 Table: {table.lower()}")
        # for col in columns:
            # print(f"    |    ├── {col.lower()}")
        print("")  # Add a blank line for readability

def print_schema(dbs_name):
    # Example usage
    print(f"{dbs_name.lower()}")

    schema_dir = "../../data/schema"
    pattern = os.path.join(schema_dir, f"{dbs_name}*.json")
    matched_files = glob.glob(pattern)

    if not matched_files:
        print(f"⚠️ No schema files found with prefix '{dbs_name}'")
        return

    schema_path = matched_files[0]
    print(f"📂 Matched schema file: {os.path.basename(schema_path)}")

    schema_info = extract_schema_info(schema_path)
    display_schema_info(schema_info)

In [4]:
print_schema("24205")
print_schema("66305")

24205
📂 Matched schema file: 24205_FynRegulationMolecularBiologyDb.json
Q18024903
FYN
    ├── 📂 Table: micrornaregulationentries

    ├── 📂 Table: generegulationdetails

    ├── 📂 Table: humanchromosome12details

66305
📂 Matched schema file: 66305_Trypanosoma_Cruzi_Orthologs_Db_663.json
Q62256153
TcCLB.507959.140
    ├── 📂 Table: gene_orthologs

    ├── 📂 Table: orthology



In [5]:
# 73720_93351
print_schema("73720")
print_schema("93351")



73720
📂 Matched schema file: 73720_SOYUZ_TM_MISSIONS_DATABASE.json
Q2705481
Soyuz-TM
    ├── 📂 Table: soyuz_tm_mission_details
    |    ├── mission_label
    |    ├── mission_description
    |    ├── cospar_id
    |    ├── spacecraft_catalog_number
    |    ├── launch_date_utc
    |    ├── launch_vehicle
    |    ├── spacecraft_instance
    |    ├── mission_patch_image
    |    ├── program_affiliation
    |    ├── preceding_mission
    |    ├── succeeding_mission
    |    ├── orbit_type
    |    ├── freebase_id
    |    ├── crew_members
    |    ├── spacecraft_model
    |    ├── orbits_completed
    |    ├── landing_date_utc
    |    ├── orbital_inclination_degrees
    |    ├── orbital_period_minutes
    |    ├── apoapsis_km
    |    ├── periapsis_km
    |    ├── spacecraft_mass_kg
    |    ├── mission_duration_seconds
    |    ├── landing_location
    |    ├── mission_operator
    |    ├── launch_site
    |    ├── mission_logo_image
    |    ├── launch_country
    |    ├── notable_eve

In [8]:
import sys
sys.path.append("..")
from model.WKDataset import WKDataset
SAMPLE_SIZE = 1000
INPUT_TXT = "/hpctmp/e1351271/wkdbs/out/sampled_ids_1000.txt"
dataset = WKDataset(schema_dir="../../data/schema", csv_base_dir="../../data/unzip")
sampled_ids = dataset.save_sampled_ids(INPUT_TXT, k=SAMPLE_SIZE, seed=42)

In [7]:
import os
import io
import sys
def capture_print_schema(db_id):
    buffer = io.StringIO()
    sys_stdout = sys.stdout
    sys.stdout = buffer
    try:
        print_schema(db_id)
    finally:
        sys.stdout = sys_stdout
    return buffer.getvalue()

directory = "/hpctmp/e1351271/wkdbs/out/qid_pairs_max_similarity"

similarity_data = []

for filename in os.listdir(directory):
    if filename.endswith(".txt"):
        file_path = os.path.join(directory, filename)
        with open(file_path, "r") as f:
            db_pair = None
            max_sim = None
            for line in f:
                if "<-->" in line:
                    db_pair = [s.strip() for s in line.strip().split("<-->")]
                elif line.startswith("Max similarity:"):
                    try:
                        max_sim = float(line.strip().split(":")[1])
                    except:
                        pass
            if db_pair and max_sim is not None:
                similarity_data.append((filename, max_sim, db_pair[0], db_pair[1]))

similarity_data.sort(key=lambda x: x[1])
top10 = similarity_data[:10]

output_dir = "least_similarity_pairs"
os.makedirs(output_dir, exist_ok=True)

for i, (fname, sim, db1, db2) in enumerate(top10):
    outname = f"{i+1:02d}_{fname}"
    outpath = os.path.join(output_dir, outname)
    with open(outpath, "w") as out:
        out.write(f"{fname}\t{sim:.4f}\n\n")
        out.write(f"===== Schema for {db1} =====\n")
        out.write(capture_print_schema(db1) + "\n")
        out.write(f"\n===== Schema for {db2} =====\n")
        out.write(capture_print_schema(db2) + "\n")

In [1]:
import os

# 源目录
directory = "/hpctmp/e1351271/wkdbs/out/random_pairs_max_similarity_10k"

# 存储提取结果
extracted = []

# 遍历所有文件
for filename in os.listdir(directory):
    if "_" in filename:
        parts = filename.split("_")
        if len(parts) >= 2:
            # 提取第二个下划线前的部分（即前两个parts合并）
            extracted.append(f"{parts[0]}_{parts[1]}")

# 去重（可选）
extracted = list(set(extracted))

# 保存到文件
with open("sample_pairs_10k.txt", "w") as f:
    for item in extracted:
        f.write(item + "\n")


In [9]:
import pandas as pd
import itertools
import csv

def convert_duplicate_qids_to_pair_csv_with_qid(
    input_csv="/hpctmp/e1351271/wkdbs/data/duplicate_qids.csv",
    output_csv="/hpctmp/e1351271/wkdbs/data/qid_pairs.csv"
):
    df = pd.read_csv(input_csv)
    pairs = []

    for _, row in df.iterrows():
        qid = row["Q-ID"]
        dbs = [db.strip().split("_")[0] for db in row["Databases"].split(";") if db.strip()]
        for db1, db2 in itertools.combinations(dbs, 2):
            print(db1, db2, qid)
            pairs.append((db1, db2, qid))  # ✅ 保留 qid

    # 去重
    unique_pairs = list(set(pairs))

    # 写入 CSV
    with open(output_csv, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["db1", "db2", "qid"])  # ✅ header 包含 qid
        for db1, db2, qid in unique_pairs:
            writer.writerow([db1, db2, qid])

    print(f"✅ Saved {len(unique_pairs)} pairs WITH QID to {output_csv}")


convert_duplicate_qids_to_pair_csv_with_qid()

98018 63035 Q16868
42581 58223 Q164735
03603 59772 Q1489
03603 27229 Q1489
59772 27229 Q1489
63835 30121 Q5968
40821 52883 Q8646
40821 92311 Q8646
40821 56849 Q8646
40821 00895 Q8646
40821 02918 Q8646
40821 65330 Q8646
40821 92081 Q8646
40821 07679 Q8646
52883 92311 Q8646
52883 56849 Q8646
52883 00895 Q8646
52883 02918 Q8646
52883 65330 Q8646
52883 92081 Q8646
52883 07679 Q8646
92311 56849 Q8646
92311 00895 Q8646
92311 02918 Q8646
92311 65330 Q8646
92311 92081 Q8646
92311 07679 Q8646
56849 00895 Q8646
56849 02918 Q8646
56849 65330 Q8646
56849 92081 Q8646
56849 07679 Q8646
00895 02918 Q8646
00895 65330 Q8646
00895 92081 Q8646
00895 07679 Q8646
02918 65330 Q8646
02918 92081 Q8646
02918 07679 Q8646
65330 92081 Q8646
65330 07679 Q8646
92081 07679 Q8646
66266 88685 Q736
66266 64857 Q736
66266 40059 Q736
88685 64857 Q736
88685 40059 Q736
64857 40059 Q736
72697 41654 Q1490
72697 07830 Q1490
72697 96274 Q1490
72697 29618 Q1490
72697 35446 Q1490
72697 92546 Q1490
41654 07830 Q1490
41654 96274 Q

In [3]:
import os
import re
import csv

INPUT_DIR = "/hpctmp/e1351271/wkdbs/out/random_pairs_max_similarity_10k"
OUTPUT_CSV = "/hpctmp/e1351271/wkdbs/out/col_matcher_cosine_idf/col_matcher_sample_10k.csv"

# 正则表达式
pair_pattern = re.compile(r"Matched pair: \('(.*)',\s*'(.*)'\)")
similarity_pattern = re.compile(r"Max similarity:\s+([0-9.]+)")
runtime_pattern = re.compile(r"Runtime:\s+([0-9.]+)\s+seconds")
db_pattern = re.compile(r"(\d{5})\s+<-->\s+(\d{5})")

records = []

for fname in os.listdir(INPUT_DIR):
    if not fname.endswith(".txt"):
        continue
    fpath = os.path.join(INPUT_DIR, fname)
    with open(fpath, "r", encoding="utf-8") as f:
        lines = f.readlines()

    # 提取 db ids 从文件名或首行
    db_match = db_pattern.search(lines[0])
    if not db_match:
        print(f"⚠️ Failed to extract db pair from {fname}")
        continue
    db1, db2 = db_match.group(1), db_match.group(2)

    # 提取 matched pair
    pair_match = None
    for line in lines:
        if "Matched pair" in line:
            pair_match = pair_pattern.search(line)
            break
    if not pair_match:
        print(f"⚠️ No matched pair found in {fname}")
        continue
    col1, col2 = pair_match.group(1), pair_match.group(2)

    # 提取 similarity
    similarity = None
    for line in lines:
        sim_match = similarity_pattern.search(line)
        if sim_match:
            similarity = float(sim_match.group(1))
            break
    if similarity is None:
        print(f"⚠️ Similarity missing in {fname}")
        continue

    # 提取 runtime
    runtime = None
    for line in lines:
        run_match = runtime_pattern.search(line)
        if run_match:
            runtime = float(run_match.group(1))
            break
    if runtime is None:
        print(f"⚠️ Runtime missing in {fname}")
        continue

    records.append([db1, db2, col1, col2, similarity, runtime])

# 写入 CSV
os.makedirs(os.path.dirname(OUTPUT_CSV), exist_ok=True)
with open(OUTPUT_CSV, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["db_1", "db_2", "column_1", "column_2", "similarity", "runtime_seconds"])
    writer.writerows(records)

print(f"✅ Converted {len(records)} files to: {OUTPUT_CSV}")



📂 Processing /hpctmp/e1351271/wkdbs/out/qid_pairs_max_similarity
⚠️ No matched pair found in 85700_60209_Q83310_pair.txt
⚠️ No matched pair found in 20698_89092_Q47715_pair.txt
⚠️ No matched pair found in 58794_61614_Q212_pair.txt
⚠️ No matched pair found in 88736_09644_Q5287_pair.txt
⚠️ No matched pair found in 65468_09644_Q5287_pair.txt
⚠️ No matched pair found in 61614_84740_Q212_pair.txt
⚠️ No matched pair found in 50271_09644_Q5287_pair.txt
⚠️ No matched pair found in 97760_61614_Q212_pair.txt
⚠️ No matched pair found in 40067_30407_Q7108_pair.txt
✅ Saved 8795 rows to /hpctmp/e1351271/wkdbs/out/col_matcher_cosine_no_idf/col_matcher_qid.csv

📂 Processing /hpctmp/e1351271/wkdbs/out/sample_pairs_max_similarity
✅ Saved 0 rows to /hpctmp/e1351271/wkdbs/out/col_matcher_cosine_no_idf/col_matcher_sample_10k.csv
